Install dependencies and libraries

In [ ]:
! pip install pytorch_tabular
! git clone https://github.com/manujosephv/pytorch_tabular
%cd pytorch_tabular

!python setup.py install
!pip install setuptools==59.5.0





In [ ]:
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False
if not IN_COLAB:
    os.chdir("..")
%load_ext autoreload
%autoreload 2

from pytorch_tabular import TabularModel
from pytorch_tabular.models import CategoryEmbeddingModelConfig, FTTransformerConfig, FTTransformerModel, TabNetModelConfig, TabNetModel, AutoIntConfig, AutoIntConfig, TabTransformerConfig, TabTransformerModel
from pytorch_tabular.models import AutoIntModel, AutoIntConfig, NodeConfig, NODEModel
from pytorch_tabular.config import DataConfig, OptimizerConfig, TrainerConfig, ExperimentConfig, ModelConfig
from pytorch_tabular.models import BaseModel

import torch
import torch.nn as nn
import torch.nn.functional as F
from omegaconf import DictConfig
from typing import Dict
from dataclasses import dataclass, field


from sklearn.preprocessing import MinMaxScaler
import plotly.express as px
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode
from sklearn.model_selection import train_test_split

# for plots
font = {'family': 'serif',
        'color':  'darkred',
        'weight': 'normal',
        'size': 16
        }

font_title = {'family': 'serif',
        'color':  'darkred',
        'weight': 'normal',
        'size': 20
        }

# Set random seed
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed);


FT-Transformer architecture

In [ ]:
plt.scatter(np.linspace(0,100, 10000), np.sin(0.1*np.linspace(0,100, 10000)) + 0.03*np.linspace(0,100, 10000))

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(np.linspace(0,100, 10000), np.sin(0.1*np.linspace(0,100, 10000)) + 0.03*np.linspace(0,100, 10000), random_state = 42)
X_val, X_test, y_val, y_test = train_test_split(X_val, y_val)

df_train = pd.DataFrame({'col1':X_train, 'cat1':0, 'target':y_train})
df_valid = pd.DataFrame({'col1':X_val, 'cat1':0, 'target':y_val})
df_test = pd.DataFrame({'col1':X_test, 'cat1':0, 'target':y_test})

In [ ]:
# FT - Transformer (new data)
num_col_names = ['col1']
cat_col_names = ['cat1']


epochs = 20
batch_size = 1000
steps_per_epoch = int((len(df_train)//batch_size)*0.9)
data_config = DataConfig(
    target=['target'],
    continuous_cols=num_col_names,
    categorical_cols=cat_col_names,
#         continuous_feature_transform="quantile_uniform"
)
trainer_config = TrainerConfig(
    auto_lr_find=False, # Runs the LRFinder to automatically derive a learning rate
    batch_size=batch_size,
    max_epochs=epochs,
    early_stopping_patience = 10,
    gpus=None,  #index of the GPU to use. -1 means all available GPUs, None, means CPU
)
# optimizer_config = OptimizerConfig(lr_scheduler="OneCycleLR", lr_scheduler_params={"max_lr":0.005, "epochs": epochs, "steps_per_epoch":steps_per_epoch})

optimizer_config = OptimizerConfig(lr_scheduler="ReduceLROnPlateau", lr_scheduler_params={"patience":10})


model_config = FTTransformerConfig(
    task = "regression",
    learning_rate=1e-4,
    seed = 42,
    input_embed_dim = 32,
    num_heads = 16,
    num_attn_blocks = 10,
    ff_dropout = 0.2,
    out_ff_layers = "1024-512-256",
    #out_ff_activation = "LeakyReLU",
    out_ff_activation = "ReLU",
    out_ff_initialization="kaiming",
    batch_norm_continuous_input=False,
    #         target_range=[(df_train[col].min(),df_train[col].max()) for col in ['target']]
)



tabular_model = TabularModel(
    data_config=data_config,
    model_config=model_config,
    optimizer_config=optimizer_config,
    trainer_config=trainer_config
)

In [ ]:
tabular_model.fit(train = df_train, validation=df_valid)

In [ ]:
pred_df = tabular_model.predict(df_test, ret_logits=False)
pred_df.head()

In [ ]:
#outputs = np.vstack([tabular_model.predict(df_test, ret_logits=False)['target_prediction'] for i in range(5)])
result = tabular_model.predict(df_test, )
result

In [ ]:
model_config = FTTransformerConfig(
    task = "regression",
    learning_rate=1e-4,
    seed = 42,
    input_embed_dim = 32,
    num_heads = 16,
    num_attn_blocks = 10,
    ff_dropout = 0.2,
    out_ff_layers = "1024-512-256",
    out_ff_activation = "LeakyReLU",
    #out_ff_activation = "ReLU",
    out_ff_initialization="kaiming",
    batch_norm_continuous_input=False,
    #         target_range=[(df_train[col].min(),df_train[col].max()) for col in ['target']]
)



tabular_model_leakeyrelu = TabularModel(
    data_config=data_config,
    model_config=model_config,
    optimizer_config=optimizer_config,
    trainer_config=trainer_config
)

tabular_model_leakeyrelu.fit(train=df_train, validation=df_valid)

pred_df_leakeyrelu = tabular_model_leakeyrelu.predict(df_test, ret_logits=False)
pred_df_leakeyrelu.head()

In [ ]:


plt.figure(figsize = (15,12))
plt.scatter(pred_df['col1'], pred_df['target_prediction'], label = 'FT-Transform prediction (with ReLU activation)')
plt.scatter(pred_df_leakeyrelu['col1'], pred_df_leakeyrelu['target_prediction'], label = 'FT-Transform prediction (with LeakeyRelu activation)')
plt.scatter(pred_df['col1'], pred_df['target'], label = 'True')
plt.xlabel("X", fontdict = font)
plt.ylabel("Y = sin(0.1X) + 0.03X", fontdict = font)
plt.title("Homoskedastic and continuous dataset", fontdict = font_title)
plt.legend(fontsize = 14);

TabNet

In [ ]:
# TabNet

epochs = 15
batch_size = 64
steps_per_epoch = int((len(df_train)//batch_size)*0.9)
data_config = DataConfig(
    target=['target'],
    continuous_cols=num_col_names,
    categorical_cols=cat_col_names,
#         continuous_feature_transform="quantile_uniform"
)
trainer_config = TrainerConfig(
    auto_lr_find=False, # Runs the LRFinder to automatically derive a learning rate
    batch_size=batch_size,
    max_epochs=epochs,
    early_stopping_patience = 5,
    gpus=None,  #index of the GPU to use. -1 means all available GPUs, None, means CPU
)
# optimizer_config = OptimizerConfig(lr_scheduler="OneCycleLR", lr_scheduler_params={"max_lr":0.005, "epochs": epochs, "steps_per_epoch":steps_per_epoch})

optimizer_config = OptimizerConfig(lr_scheduler="ReduceLROnPlateau", lr_scheduler_params={"patience":3})


model_config = TabNetModelConfig(
    task = "regression"
)



tabular_model_tabnet = TabularModel(
    data_config=data_config,
    model_config=model_config,
    optimizer_config=optimizer_config,
    trainer_config=trainer_config
)

tabular_model_tabnet.fit(train=df_train, validation=df_valid)

In [ ]:
pred_df_tabnet = tabular_model_tabnet.predict(df_test, ret_logits=False)
pred_df_tabnet.head()

In [ ]:
plt.figure(figsize = (15,12))
plt.scatter(pred_df['col1'], pred_df['target_prediction'], label = 'FT-Transform prediction (with ReLU activation)', color = 'purple')
plt.scatter(pred_df_leakeyrelu['col1'], pred_df_leakeyrelu['target_prediction'], label = 'FT-Transform prediction (with LeakeyRelu activation)', color = 'blue')
plt.scatter(pred_df_tabnet['col1'], pred_df_tabnet['target_prediction'], label = 'TabNet prediction', color = 'green')
plt.scatter(pred_df['col1'], pred_df['target'], label = 'True', color = 'black')
plt.xlabel("X", fontdict = font)
plt.ylabel("Y = sin(0.1X) + 0.03X", fontdict = font)
plt.title("Homoskedastic and continuous dataset", fontdict = font_title)
plt.legend(fontsize = 14);

Node

In [ ]:
# Node

epochs = 15
batch_size = 64
steps_per_epoch = int((len(df_train)//batch_size)*0.9)
data_config = DataConfig(
    target=['target'],
    continuous_cols=num_col_names,
    categorical_cols=cat_col_names,
#         continuous_feature_transform="quantile_uniform"
)
trainer_config = TrainerConfig(
    auto_lr_find=False, # Runs the LRFinder to automatically derive a learning rate
    batch_size=batch_size,
    max_epochs=epochs,
    early_stopping_patience = 5,
    gpus=None,  #index of the GPU to use. -1 means all available GPUs, None, means CPU
)
# optimizer_config = OptimizerConfig(lr_scheduler="OneCycleLR", lr_scheduler_params={"max_lr":0.005, "epochs": epochs, "steps_per_epoch":steps_per_epoch})

optimizer_config = OptimizerConfig(lr_scheduler="ReduceLROnPlateau", lr_scheduler_params={"patience":3})


model_config = NodeConfig(
    task = "regression"
)



tabular_model_node = TabularModel(
    data_config=data_config,
    model_config=model_config,
    optimizer_config=optimizer_config,
    trainer_config=trainer_config
)

tabular_model_node.fit(train=df_train, validation=df_valid)
pred_df_node = tabular_model_node.predict(df_test, ret_logits=False)

In [ ]:
plt.figure(figsize = (15,12))
plt.scatter(pred_df['col1'], pred_df['target_prediction'], label = 'FT-Transform prediction (with ReLU activation)', color = 'purple')
plt.scatter(pred_df_leakeyrelu['col1'], pred_df_leakeyrelu['target_prediction'], label = 'FT-Transform prediction (with LeakeyRelu activation)', color = 'blue')
plt.scatter(pred_df_tabnet['col1'], pred_df_tabnet['target_prediction'], label = 'TabNet prediction', color = 'green')
plt.scatter(pred_df_node['col1'], pred_df_node['target_prediction'], label = 'Node prediction', color = 'red')
plt.scatter(pred_df['col1'], pred_df['target'], label = 'True', color = 'black')
plt.xlabel("X", fontdict = font)
plt.ylabel("Y = sin(0.1X) + 0.03X", fontdict = font)
plt.title("Hohoskedastic and continuous dataset", fontdict = font_title)
plt.legend(fontsize = 14);

TabTransformer

In [ ]:
# TabTransformer


epochs = 15
batch_size = 64
steps_per_epoch = int((len(df_train)//batch_size)*0.9)
data_config = DataConfig(
    target=['target'],
    continuous_cols=num_col_names,
    categorical_cols=cat_col_names,
#         continuous_feature_transform="quantile_uniform"
)
trainer_config = TrainerConfig(
    auto_lr_find=False, # Runs the LRFinder to automatically derive a learning rate
    batch_size=batch_size,
    max_epochs=epochs,
    early_stopping_patience = 5,
    gpus=None,  #index of the GPU to use. -1 means all available GPUs, None, means CPU
)
# optimizer_config = OptimizerConfig(lr_scheduler="OneCycleLR", lr_scheduler_params={"max_lr":0.005, "epochs": epochs, "steps_per_epoch":steps_per_epoch})

optimizer_config = OptimizerConfig(lr_scheduler="ReduceLROnPlateau", lr_scheduler_params={"patience":3})


model_config = TabTransformerConfig(
    task = "regression"
)



tabular_model_tabtransformer = TabularModel(
    data_config=data_config,
    model_config=model_config,
    optimizer_config=optimizer_config,
    trainer_config=trainer_config
)

tabular_model_tabtransformer.fit(train=df_train, validation=df_valid)
pred_df_tabtransformer = tabular_model_tabtransformer.predict(df_test, ret_logits=False)


In [ ]:
plt.figure(figsize = (15,12))
plt.scatter(pred_df['col1'], pred_df['target_prediction'], label = 'FT-Transform prediction (with ReLU activation)', color = 'purple')
plt.scatter(pred_df_leakeyrelu['col1'], pred_df_leakeyrelu['target_prediction'], label = 'FT-Transform prediction (with LeakeyRelu activation)', color = 'blue')
plt.scatter(pred_df_tabnet['col1'], pred_df_tabnet['target_prediction'], label = 'TabNet prediction', color = 'green')
plt.scatter(pred_df_node['col1'], pred_df_node['target_prediction'], label = 'Node prediction', color = 'red')
plt.scatter(pred_df_tabtransformer['col1'], pred_df_tabtransformer['target_prediction'], label = 'TabTransformer prediction', color = 'yellow')
plt.scatter(pred_df['col1'], pred_df['target'], label = 'True', color = 'black')
plt.xlabel("X", fontdict = font)
plt.ylabel("Y = sin(0.1X) + 0.03X", fontdict = font)
plt.title("Hohoskedastic and continuous dataset", fontdict = font_title)
plt.legend(fontsize = 14);

AutoInt

In [ ]:
# AutoInt

epochs = 15
batch_size = 64
steps_per_epoch = int((len(df_train)//batch_size)*0.9)
data_config = DataConfig(
    target=['target'],
    continuous_cols=num_col_names,
    categorical_cols=cat_col_names,
#         continuous_feature_transform="quantile_uniform"
)
trainer_config = TrainerConfig(
    auto_lr_find=False, # Runs the LRFinder to automatically derive a learning rate
    batch_size=batch_size,
    max_epochs=epochs,
    early_stopping_patience = 5,
    gpus=None,  #index of the GPU to use. -1 means all available GPUs, None, means CPU
)
# optimizer_config = OptimizerConfig(lr_scheduler="OneCycleLR", lr_scheduler_params={"max_lr":0.005, "epochs": epochs, "steps_per_epoch":steps_per_epoch})

optimizer_config = OptimizerConfig(lr_scheduler="ReduceLROnPlateau", lr_scheduler_params={"patience":3})


model_config = AutoIntConfig(
    task = "regression"
)



tabular_model_autoint = TabularModel(
    data_config=data_config,
    model_config=model_config,
    optimizer_config=optimizer_config,
    trainer_config=trainer_config
)

tabular_model_autoint.fit(train=df_train, validation=df_valid)



In [ ]:
pred_df_autoint = tabular_model.predict(df_test, ret_logits=False)
pred_df_autoint.head()



In [ ]:
plt.figure(figsize = (15,12))
plt.scatter(pred_df['col1'], pred_df['target_prediction'], label = 'FT-Transform prediction (with ReLU activation)', color = 'purple')
plt.scatter(pred_df_leakeyrelu['col1'], pred_df_leakeyrelu['target_prediction'], label = 'FT-Transform prediction (with LeakeyRelu activation)', color = 'blue')
plt.scatter(pred_df_tabnet['col1'], pred_df_tabnet['target_prediction'], label = 'TabNet prediction', color = 'green')
#plt.scatter(pred_df_node['col1'], pred_df_node['target_prediction'], label = 'Node prediction', color = 'red')
plt.scatter(pred_df_tabtransformer['col1'], pred_df_tabtransformer['target_prediction'], label = 'TabTransformer prediction', color = 'yellow')
plt.scatter(pred_df_autoint['col1'], pred_df_autoint['target_prediction'], label = 'AutoInt prediction', color = 'pink')
plt.scatter(pred_df['col1'], pred_df['target'], label = 'True', color = 'black')
plt.xlabel("X", fontdict = font)
plt.ylabel("Y = sin(0.1X) + 0.03X", fontdict = font)
plt.title("Hohoskedastic and continuous dataset", fontdict = font_title)
plt.legend(fontsize = 14);

In [ ]:
def uncertainity_estimate(x, model, num_samples, l2):
    outputs = np.hstack([model.predict(x, ret_logits=False).detach().numpy() for i in range(num_samples)]) # n번 inference, output.shape = [20, N]
    y_mean = outputs.mean(axis=1)
    y_variance = outputs.var(axis=1)
    tau = l2 * (1. - model.dropout_rate) / (2. * N * model.decay)
    y_variance += (1. / tau)
    y_std = np.sqrt(y_variance)
    return y_mean, y_std


# Normalise data:

x_mean, x_std = df_train['col1'].mean(), df_train['col1'].std()
y_mean, y_std = df_train['target'].mean(), df_train['target'].std()
x_obs = (df_train['col1'] - x_mean) / x_std
y_obs = (df_train['target'] - y_mean) / y_std
x_test = (df_valid['col1'] - x_mean) / x_std
y_test = (df_valid['target'] - y_mean) / y_std

iters_uncertainty = 200

lengthscale = 0.01

device = torch.device("cuda")
n_std = 2 # number of standard deviations to plot
y_mean, y_std = uncertainity_estimate(x = torch.Tensor(x_test).view(-1,1).to(device),
                                      model = tabular_model, iters_uncertainty, lengthscale)

plt.figure(figsize=(12,6))
plt.plot(x_obs, y_obs, ls="none", marker="o", color="0.1", alpha=0.8, label="observed")
plt.plot(x_test, y_mean, ls="-", color="b", label="mean")
plt.plot(x_test, y_test, ls='--', color='r', label='true')
for i in range(n_std):
    plt.fill_between( x_test,
        y_mean - y_std * ((i+1.)),
        y_mean + y_std * ((i+1.)),
        color="b",
        alpha=0.1)
plt.legend()
plt.grid()

In [ ]:
def uncertainity_estimate(x, model, num_samples, l2):
    outputs = np.hstack([model(x).cpu().detach().numpy() for i in range(num_samples)]) # n번 inference, output.shape = [20, N]
    y_mean = outputs.mean(axis=1)
    y_variance = outputs.var(axis=1)
    tau = l2 * (1. - model.dropout_rate) / (2. * N * model.decay)
    y_variance += (1. / tau)
    y_std = np.sqrt(y_variance)
    return y_mean, y_std